In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        
        print(os.path.join(dirname, filename))

        
        

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    Trainer,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "bigscience/bloomz-7b1"

# The instruction dataset to use
dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
new_model = "bigscience/bloomz-7b1"
# LoRA attention dimension
lora_r = 64
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

use_nested_quant = False

output_dir = "./results"

# Number of training epochs
num_train_epochs = 1
fp16 = False
bf16 = False

per_device_train_batch_size = 1

per_device_eval_batch_size = 4

gradient_accumulation_steps = 4

gradient_checkpointing = True

max_grad_norm = 0.3

learning_rate = 2e-4

weight_decay = 0.001

optim = "adamw_torch"


max_steps = -1
warmup_ratio = 0.03
group_by_length = True

save_steps = 0

logging_steps = 25

max_seq_length = False
packing = False
#device_map = {"": 0}

In [ ]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
import csv

data=pd.read_csv("/kaggle/input/expect-answer-true-or-false-arabic/data.csv")

In [ ]:
import pandas as pd
train_df = data.iloc[:-3000]
valid_df = data.iloc[-3000:]
train_df['input']= f"\n\ncontext: "+train_df['context']+"/n/nquestion"+train_df['question']+ f".\n\nstudent: "+train_df['answer']+f" \n\n##response : "+train_df['label']
valid_df['input']=f"\n\ncontext: "+valid_df['context']+"/n/nquestion"+valid_df['question']+ f".\n\nstudent: "+valid_df['answer']+f" \n\n##response : "+valid_df['label']
valid_df['input2']=f"\n\ncontext: "+valid_df['context']+"/n/nquestion"+valid_df['question']+ f".\n\nstudent: "+valid_df['answer']+f" \n\n##response : "

In [ ]:
tokenizer.padding_side = "right"

In [ ]:
m=-12
w=np.zeros(len(train_df))
o=0
for i in train_df['input']:
    t=len(tokenizer(i)['input_ids'])
    w[o]=t
    o+=1
m=-12
a=np.zeros(len(valid_df))
o=0
for i in valid_df['input']:
    t=len(tokenizer(i)['input_ids'])
    a[o]=t
    o+=1    
train_df=train_df.loc[w<600]
valid_df=valid_df.loc[a<600]   
train_texts=train_df
valid_texts=valid_df

In [ ]:
#Datasets and Dataloaders
from torch.utils.data import Dataset, DataLoader

class QADataset(Dataset):
    def __init__(self, encodings):
        self.inputs = encodings['input']
        
    def __getitem__(self, idx):
        a=tokenizer(self.inputs[idx] , truncation=True, padding='max_length', return_tensors="pt", max_length=600)
        return {
            
            "input_ids": a["input_ids"][0],
            "attention_mask": a["attention_mask"][0],
            "labels":a['input_ids'][0]
        }
    def __len__(self):
        return len(self.inputs)

train_dataset = QADataset(train_texts.iloc[10000:20000].reset_index(drop=True))
val_dataset = QADataset(valid_texts.iloc[:100].reset_index(drop=True))


In [ ]:

!pip install wandb
import wandb
wandb.login(key="14459c516497ab76a78f7fc1278bfe60d301d250")

In [ ]:
train_texts.shape

In [ ]:
peft= PeftModel.from_pretrained(model,"/kaggle/input/bloomz-7b1-ar/results/checkpoint-2400",is_trainable=True)
peft.enable_input_require_grads ()
#peft=model

In [ ]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=300,
    logging_steps=5,
    learning_rate=2e-4,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=False,
    lr_scheduler_type='constant',
)
trainer = SFTTrainer(
    model=peft,
        peft_config=None,

    train_dataset=train_dataset,
        eval_dataset=val_dataset,
    dataset_text_field="text",
    args=training_arguments,
    packing=False,
)
trainer.train()
trainer.model.save_pretrained(new_model)

In [ ]:
peft.save_pretrained("bloom")


In [ ]:
trainer.evaluate()


In [ ]:
! pip install evaluate


In [ ]:
! pip install rouge_score

In [ ]:

tokenizer.padding_side = "left"


In [ ]:
peft.eval()

In [ ]:
from transformers import logging

# Disable transformers library warnings
logging.set_verbosity_error()
import evaluate
import numpy as np
from datasets import load_from_disk
from tqdm import tqdm

# Metric
metric= evaluate.load("rouge")
metric2= evaluate.load("bleu")

predictions, references = [] , []
o=0
s=0
l=1000
step=8
for i in range(0,l,step):
        inp2=valid_df['input2'].iloc[i:i+step]
        w=tokenizer(inp2.tolist(), add_special_tokens=True,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',    max_length=600

        )
        d=tokenizer.batch_decode(peft.generate(input_ids=w['input_ids'].cuda(),attention_mask=w['attention_mask'].cuda()),skip_special_tokens=True)
        for o in range(len(d)):                         
            e=d[o][d[o].find(f'\n\n##response : ')+len(f'\n\n##response : '):]
            a=valid_df['label'].iloc[i+o].split()[0]
            print(e,valid_df['label'].iloc[i+o].split()[0].strip().lower())
            s+=int((e+' ,').split()[0].strip().lower()==valid_df['label'].iloc[i+o].split()[0].strip().lower())
            print(f'{i} : {s/(i+o+1)} ',end='\r')
            predictions+=[e]
            references+=[valid_df['label'].iloc[i+o]]


In [ ]:
print(f"accuracy : {s/l }")

In [ ]:
peft_model_id="bloom"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)